<a href="https://colab.research.google.com/github/atishaye/Video-Recommendation-System/blob/main/Google%20Colab%20Notebooks/TigerGraph_Extern_Dashboard_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Utilities and Constants

## 1.1 Imports and Downloads 

In [ ]:
!pip install -q jupyter-dash
!pip install -q pyTigerGraph

!pip install -q dash-bootstrap-components
!pip install -q dash_daq
!pip install dash-extensions
!pip install dash-loading-spinners
!pip install visdcc
!pip install dash-cytoscape

from jupyter_dash import JupyterDash
import dash     
import dash_table
import dash_cytoscape as cyto
from dash.exceptions import PreventUpdate
import dash_core_components as dcc
import dash_html_components as html
import dash_loading_spinners as dls
import dash_daq as daq
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go

import pyTigerGraph as tg
import pandas as pd
import pprint
import json
import visdcc


from sortedcontainers import SortedDict
from statistics import mean
import random
import time
import os

cyto.load_extra_layouts()

In [2]:
!git clone https://github.com/atishaye/Video-Recommendation-System.git

Cloning into 'Video-Recommendation-System'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 176 (delta 46), reused 29 (delta 5), pack-reused 72
Receiving objects: 100% (176/176), 168.74 MiB | 29.78 MiB/s, done.
Resolving deltas: 100% (58/58), done.
Checking out files: 100% (46/46), done.


## 1.2 TigerGraph Connection

In [ ]:
TG_SUBDOMAIN = 'tiger-graph' 
TG_HOST = "https://" + TG_SUBDOMAIN + ".i.tgcloud.io" # GraphStudio Link

TG_USERNAME = "tigergraph"
TG_PASSWORD = "tigergraph"
TG_GRAPHNAME = "Demo"

conn = tg.TigerGraphConnection(host=TG_HOST, graphname=TG_GRAPHNAME, username=TG_USERNAME, password=TG_PASSWORD)
conn.apiToken = conn.getToken(conn.createSecret())

conn.gsql('ls')

## 1.3 Pretty Print

In [4]:
def pprint(string):
  print(json.dumps(string, indent=2))

## 1.4 Constants

In [5]:
# Theme colors utilized throughout the Dashboard

TG_YELLOW = "#FFCF9E"
TG_ORANGE = "#f5bd1f"
WHITE = "#fffff8"
BAR_LIGHTRED = "#ed8d84"

MAINSTYLE = {
  "background-color": "#FFFF99",
  'font': {'family': 'Roboto'},
}

# Links to logos and fonstyles 

TG_LOGO = "https://avatars.githubusercontent.com/u/71526309?s=280&v=4"
DASH_LOGO = "https://rapids.ai/assets/images/Plotly_Dash_logo.png"
PLOTLY_LOGO = "https://dash.plotly.com/docs/assets/images/dark_plotly_dash_logo.png"

# Used for genre tags (ensuring that each is a distinc color)
COLOR_DICT = {}
colors = ["#f8c205" "#f8c2bc", "#f8c2e6", "#79c2e6", "#9c83e6", "#9ccf2d", "#ff63bf", "#92b9bf", "#ef3c8d", "#50f59e", "#5c64ec", "#ed4e84", "#32d9de", "#327bde"]

# 2. Queries and Data Retrieval

## 2.1 Flatten Query

In [ ]:
#TO VERTEX Topic VALUES ($0, _),
#TO VERTEX Topic VALUES ($0, $1, _),
res = conn.gsql('''
    USE GRAPH Demo
    BEGIN
    CREATE LOADING JOB load_flattened_topic_keywords FOR GRAPH Demo {
    DEFINE FILENAME f;
    LOAD f
    TO TEMP_TABLE t1(dominant_topic,topic_keywords) VALUES ($1, flatten($2,"|",1)) USING QUOTE="double", SEPARATOR=",", HEADER="true", EOL="\\n";

    LOAD TEMP_TABLE t1
        TO VERTEX Keyword VALUES($"topic_keywords"),
        TO EDGE has_topic_keyword VALUES($"dominant_topic", $"topic_keywords");
    }
    END
    ''')
print(res)

## 2.2 Run Flatten Query

In [7]:
# Load the topic_keywords_file with the 'load_topic_keywords' job
topic_keywords_file = '/content/Video-Recommendation-System/Data/Final Dashboard/topics.csv'
results = conn.uploadFile(topic_keywords_file, fileTag="f", jobName="load_flattened_topic_keywords")
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 26,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  },
  {
    "tempTableName": "t1",
    "sourceFile": "Online_POST",
    "statistics": {
      "vertex": [
        {
          "typeName": "Keyword",
          "validObject": 751,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [
        {
          "typeName": "has_topic_keyword",
          "validObject": 751,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 1,
          "invalidSecondaryId": 0,
          "incorrectFixedBi

## 2.3 Run timestamp_list query

In [ ]:
results = conn.gsql('''
  USE GRAPH Demo
  BEGIN
  CREATE LOADING JOB l1 FOR GRAPH Demo {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Keyword VALUES($0) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  LOAD MyDataSource TO EDGE has_keyword VALUES($0, $1, SPLIT($2,"|")) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  }
  END
  ''')
print(results)

In [9]:
# Load the posts file wiht the 'load_posts' job
keyword_with_timestamps_file = '/content/Video-Recommendation-System/Data/Final Dashboard/keywords_with_timestamps.csv'
results = conn.uploadFile(keyword_with_timestamps_file, fileTag='MyDataSource', jobName='l1')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 1941,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Keyword",
          "validObject": 1941,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [
        {
          "typeName": "has_keyword",
          "validObject": 1941,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 1,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


## 2.4 Loading Transcript set

In [10]:
results = conn.gsql('''
  USE GRAPH Demo
  BEGIN
  CREATE LOADING JOB load_transcript_set FOR GRAPH Demo {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Transcript VALUES($0, SPLIT($2, "|"), $3) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  LOAD MyDataSource TO EDGE has_transcript VALUES($0, $0) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  }
  END
  ''')
print(results)

Using graph 'Demo'
Semantic Check Fails: The job name load_transcript_set already exists in other objects!
Failed to create loading jobs: [load_transcript_set].


In [11]:
transcript_set = '/content/Video-Recommendation-System/Data/Final Dashboard/out.csv'
results = conn.uploadFile(transcript_set, fileTag='MyDataSource', jobName='load_transcript_set')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 26,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Transcript",
          "validObject": 25,
          "noIdFound": 1,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [
        {
          "typeName": "has_transcript",
          "validObject": 25,
          "noIdFound": 1,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


## 2.5 Sentiment Query and Testing

In [12]:
def get_sentiments(id = 5):
    results = conn.getVerticesById("Sentiment", str(id))
    print(results)
    def senti(id):
        return results[0]['attributes']['positive_sentiment'], results[0]['attributes']['negative_sentiment'], results[0]['attributes']['neutral_sentiment'], results[0]['attributes']['compound_sentiment']
    sentiments = {"positive_sentiment": 0, "negative_sentiment": 0, "neutral_sentiment": 0, "compound_sentiment": 0}
    sentiments["positive_sentiment"], sentiments["negative_sentiment"], sentiments["neutral_sentiment"], sentiments["compound_sentiment"] = senti(id)
    return sentiments

In [13]:
conn.getVerticesById("Sentiment", 5)

[{'attributes': {'compound_sentiment': 0.9439,
   'id': '5',
   'negative_sentiment': 0.111,
   'neutral_sentiment': 0.688,
   'positive_sentiment': 0.201},
  'v_id': '5',
  'v_type': 'Sentiment'}]

## 2.6 Get transcript_words[List]

In [14]:
def get_num_words(id=5):
    words = conn.getVerticesById("Transcript", str(id))
    return len(words[0]['attributes']['Preprocessed_Text'])

get_num_words(14)

301

In [15]:
words = conn.getVerticesById("Transcript", 5)

In [16]:
len(words[0]['attributes']['Preprocessed_Text'])

173

## 2.7 Get_title

In [17]:
def get_title(id=14):
    titles = conn.getVerticesById("Video_Audio", str(id))
    return titles[0]['attributes']['Title']

get_title(14)

'mammoth'

## 2.8 Make Title (Testing)

In [18]:
def make_title(title):
    return ' '.join(title.split('_')).title()

In [19]:
title = conn.getVerticesById("Video_Audio", 5)
pprint(title)

[
  {
    "v_id": "5",
    "v_type": "Video_Audio",
    "attributes": {
      "id": 5,
      "Title": "crypto_means",
      "Youtube_Link": "https://www.youtube.com/embed/tSSApsy7cpw"
    }
  }
]


## 2.9 Keyword --> VideoId

In [20]:
result = """
CREATE QUERY Keyword_to_VideoID(VERTEX<Keyword>k) FOR GRAPH Demo {
    MapAccum<STRING, ListAccum<STRING>> @@vidName;
    MapAccum<STRING, INT> @@vidId;
    keyword = {k};
    video_id = SELECT tgt
        FROM keyword:key - (<has_keyword:key_stamp) -:tgt
        ACCUM @@vidName += (tgt.Title -> key_stamp.start_timestamp_list)
        POST_ACCUM @@vidId += (tgt.Title -> tgt.id);
    
    PRINT @@vidId;
    PRINT @@vidName;
  }
#.INSTALL QUERY Keyword_to_VideoID
"""
print(conn.gsql(result, options=[]))

Semantic Check Fails: The query name Keyword_to_VideoID is used by another object! Please use a different name.
Failed to create queries: [Keyword_to_VideoID].


## 2.10 YT Link Integrations

In [21]:
keyword_to_video = conn.runInstalledQuery("Keyword_to_VideoID", params={'k': 'small'})
keyword_to_video_id = keyword_to_video[0]['@@vidId']
yt_link = conn.getVerticesById("Video_Audio", [str(keyword_to_video_id['chimp']), 1, 2])
pprint(yt_link)
yt_link = str(yt_link[0]['attributes']['Youtube_Link'])+'?start='+str(131)
#f'Output: {yt_link}'

[
  {
    "v_id": "2",
    "v_type": "Video_Audio",
    "attributes": {
      "id": 2,
      "Title": "chimp",
      "Youtube_Link": "https://www.youtube.com/embed/i-OaYYNdoQ8"
    }
  },
  {
    "v_id": "1",
    "v_type": "Video_Audio",
    "attributes": {
      "id": 1,
      "Title": "white_dwarf",
      "Youtube_Link": "https://www.youtube.com/embed/L48Jr4Cp594"
    }
  },
  {
    "v_id": "2",
    "v_type": "Video_Audio",
    "attributes": {
      "id": 2,
      "Title": "chimp",
      "Youtube_Link": "https://www.youtube.com/embed/i-OaYYNdoQ8"
    }
  }
]


In [22]:
def display_rec(keyword_to_video_id):
    rec = conn.runInstalledQuery("get_recommendation", params={'v_id':keyword_to_video_id})
    vid_arr = [i['Video_Audio'] for i in rec[0]['@@topResults']]
    res = conn.getVerticesById("Video_Audio", vid_arr)
    yt_link = []
    for link in res:
        yt_link.append(link['attributes']['Youtube_Link'])
    return yt_link
pprint(display_rec('1'))

[
  "https://www.youtube.com/embed/isC2Nxg0m6w",
  "https://www.youtube.com/embed/L1EV6vAmlSA",
  "https://www.youtube.com/embed/I0N67_nnaGA"
]


## 2.11 Getting all keywords to populate first dropdown

In [23]:
results = conn.gsql('''
  USE GRAPH Demo
  CREATE QUERY all_keywords_with_timestamp() FOR GRAPH Demo SYNTAX v2 { 

  keywords_with_timestamp = SELECT k FROM Video_Audio:va - (has_keyword>) - Keyword:k;
  

  PRINT keywords_with_timestamp;
  }
  ''')
pprint(results)

"Using graph 'Demo'\nSemantic Check Fails: The query name all_keywords_with_timestamp is used by another object! Please use a different name.\nFailed to create queries: [all_keywords_with_timestamp]."


In [ ]:
results = conn.runInstalledQuery("all_keywords_with_timestamp")
pprint(results)

In [ ]:
pprint(results[0])

In [26]:
df3 = conn.vertexSetToDataFrame(results[0]["keywords_with_timestamp"])
df3.head()

,v_id,id
0,eat,eat
1,walk,walk
2,coin,coin
3,weigh ten,
4,publishing,


In [ ]:
pprint(results[0]["keywords_with_timestamp"])

In [ ]:
df = conn.vertexSetToDataFrame(results[0]["keywords_with_timestamp"])
all_keywords_with_timestamps_list=list(df['v_id'])
all_keywords_with_timestamps_list=sorted(all_keywords_with_timestamps_list)
all_keywords_with_timestamps_list

## 2.12 Get Time for Duration

In [29]:
def get_time(id):
    sec = conn.runInstalledQuery("get_video_length", params={"vid": str(id)})[0]["max(@@timestampsINT)"]
    res = time.strftime("%M:%S", time. gmtime(sec))
    return str(res)

## 2.13 Get Cytoscape Data

In [ ]:
def get_cyto_data(id = 1):
    res = conn.runInstalledQuery("video_to_topic", params={"vid": str(id)})
    #pprint(res)
    data = []
    keywords = set()
    x = str(res[0]["@@t_id"][0])

    for i in res[1]['@@v_id']:
        data.append(("Video #"+str(i), "Topic #" + x))

    for i in res[2]["@@t_keywords"]:
        for j in res[2]["@@t_keywords"][i]:
            data.append(("Video #"+str(j), i))
        data.append(("Topic #"+x, i))
        

    print(data)
    return data
get_cyto_data()

## 2.14 Sentiment Dash Function

In [31]:
def Dash_GetSentiment(id=1):

  # Ensure that the input passed in is a valid integer

  if id is None: 
    return html.P("Enter a video/audio id number to begin!")
  
  try: 
    id = int(id)
  except: 
    return html.P("Error! Entered number must be integer!")

  
  # Retrieving information using pyTigerGraph and the functions from above 
  sentiments = {"positive_sentiment": 0, "negative_sentiment": 0, "neutral_sentiment": 0, "compound_sentiment": 0}
  sentiments["positive_sentiment"], sentiments["negative_sentiment"], sentiments["neutral_sentiment"], sentiments["compound_sentiment"] = get_sentiments(id)
  displayTitle = id 

## 2.15 Visualizations Page Dropdown Data

In [ ]:
tdf1 = conn.getVerticesById("Video_Audio", [str(i) for i in range (25)])

# pprint(tdf1)
df3 = conn.vertexSetToDataFrame(tdf1)
list1=df3['v_id']
list2=df3['Title']
# print(list1,list2)
vis_page_dropdown_list=df3.set_index('Title').to_dict()['v_id']
rev_vis_page_dropdown_list=df3.set_index('v_id').to_dict()['Title']
display(vis_page_dropdown_list)
display(rev_vis_page_dropdown_list)
list(vis_page_dropdown_list.keys())

## 2.16 Get Themes

In [33]:
# themes = {'video_id' : 'dominant_topic_theme'}

try:
    themes_csv = pd.read_csv("/content/Video-Recommendation-System/Data/Final Dashboard/themes.csv")
    themes = {}
    for i in range(len(themes_csv)):
        x, y = themes_csv.loc[i, 'Video/Audio Id'], themes_csv.loc[i, 'Theme']
        themes[str(x)] = y
except:
    themes = {'0': 'literature', '1': 'white dwarf', '2': 'animal kingdom', '3': 'bitcoin', '4': 'covid effect', '5': 'crypto', '6': 'cryptocurrency', '7': 'crypto', '8': 'wildlife', '9': 'wildlife', '10': 'wildlife', '11': ' market', '12': 'finance', '13': 'literature', '14': 'animals', '15': 'japanese author', '16': 'market', '17': 'literature', '18': 'wildlife', '19': 'books', '20': 'authors', '21': 'wildlife', '22': 'literature', '23': 'covid effect', '24': 'wildlife'}
print(themes)
print(sorted(list(themes.keys())))

{'0': 'literature', '1': 'white dwarf', '2': 'animal kingdom', '3': 'bitcoin', '4': 'covid effect', '5': 'crypto', '6': 'cryptocurrency', '7': 'crypto', '8': 'wildlife', '9': 'wildlife', '10': 'wildlife', '11': ' market', '12': 'finance', '13': 'literature', '14': 'animals', '15': 'japanese author', '16': 'market', '17': 'literature', '18': 'wildlife', '19': 'books', '20': 'authors', '21': 'wildlife', '22': 'literature', '23': 'covid effect', '24': 'wildlife'}
['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '3', '4', '5', '6', '7', '8', '9']


## 2.17 Add LDA Externally for iFrame

In [34]:
!gdown --id 1UD9jgZRr_l7aWOkMv_LTIO5YYQsBIunz
import os
os.makedirs('assets',exist_ok=True)
!mv lda.html ./assets/lda.html

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1UD9jgZRr_l7aWOkMv_LTIO5YYQsBIunz
To: /content/lda.html
100% 107k/107k [00:00<00:00, 73.1MB/s]


## 2.18 NER Query


In [35]:
import spacy
from spacy import displacy

DEFAULT_LABEL_COLORS = {
    "ORG": "#7aecec",
    "PRODUCT": "#bfeeb7",
    "GPE": "#feca74",
    "LOC": "#ff9561",
    "PERSON": "#aa9cfc",
    "NORP": "#c887fb",
    "FAC": "#9cc9cc",
    "EVENT": "#ffeb80",
    "LAW": "#ff8197",
    "LANGUAGE": "#ff8197",
    "WORK_OF_ART": "#f0d0ff",
    "DATE": "#bfe1d9",
    "TIME": "#bfe1d9",
    "MONEY": "#e4e7d2",
    "QUANTITY": "#e4e7d2",
    "ORDINAL": "#e4e7d2",
    "CARDINAL": "#e4e7d2",
    "PERCENT": "#e4e7d2",
}

def entname(name):
    return html.Span(name, style={
        "font-size": "0.8em",
        "font-weight": "bold",
        "line-height": "1",
        "border-radius": "0.35em",
        "text-transform": "uppercase",
        "vertical-align": "middle",
        "margin-left": "0.5rem"
    })


def entbox(children, color):
    return html.Mark(children, style={
        "background": color,
        "padding": "0.45em 0.6em",
        "margin": "0 0.25em",
        "line-height": "1",
        "border-radius": "0.35em",
    })


def entity(children, name):
    if type(children) is str:
        children = [children]

    children.append(entname(name))
    color = DEFAULT_LABEL_COLORS[name]
    return entbox(children, color)


def render(doc):
    children = []
    last_idx = 0
    for ent in doc.ents:
        children.append(doc.text[last_idx:ent.start_char])
        children.append(
            entity(doc.text[ent.start_char:ent.end_char], ent.label_))
        last_idx = ent.end_char
    children.append(doc.text[last_idx:])
    return children
text="amid longer start render color gradually hot supported beyond time four cold approximately limit chorus rotate dance extremely cause pressure collapse gravitational heat mean acre result yield undergo long radiation material hence physic helium form can currently non low siris binary eight hundred may know star oxygen six neon place third whose last among take reaction unable residual neutron small black mostly become stellar fusion radiate call coin titan nearest crystalline mass system point sufficient remnant recognize degenerate dwarf seal faint magnesium compose also think evolutionary begin white component electron ninety earth two sun volume state dense fuse comparable emission year case energy nineteen solar luminosity serious seven app source enough unusual one fitness generate twenty core first cool ten name william law away final usually carbon initially masse not support high hole maximum include percent knox matter light near milky progenitor five way sama mold come temperature significant edge "

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)


## 2.19 NER Side Bar Graph Utils

In [36]:
NER = spacy.load("en_core_web_sm")
def NERUtils(text):
  text1= NER(text)
  d={}
  for word in text1.ents:
      # print(word.text,word.label_)
      if word.label_ not in list(d.keys()):
        d[word.label_]=1
      else:
        d[word.label_]+=1
  return d

In [37]:
# TRANSCRIPT FOR NER
transcripts = conn.runInstalledQuery("get_transcripts", params={'vid': '1'})
# pprint(transcripts)
df6 = conn.vertexSetToDataFrame(transcripts[0]['transcript'])
# display(df6)
transcript=""
transcript_keywords=list(df6['Preprocessed_Text'])
# display(transcript_keywords[0])
for i in range(len(transcript_keywords[0])):
  transcript += str(transcript_keywords[0][i]) + ' '
print(transcript)

amid longer start render color gradually hot supported beyond time four cold approximately limit chorus rotate dance extremely cause pressure collapse gravitational heat mean acre result yield undergo long radiation material hence physic helium form can currently non low siris binary eight hundred may know star oxygen six neon place third whose last among take reaction unable residual neutron small black mostly become stellar fusion radiate call coin titan nearest crystalline mass system point sufficient remnant recognize degenerate dwarf seal faint magnesium compose also think evolutionary begin white component electron ninety earth two sun volume state dense fuse comparable emission year case energy nineteen solar luminosity serious seven app source enough unusual one fitness generate twenty core first cool ten name william law away final usually carbon initially masse not support high hole maximum include percent knox matter light near milky progenitor five way sama mold come temper

# 3. Dashboard Styling | CSS

In [38]:
# Resources Styles
RESOURCES_STYLES={
    "margin": "0",
    "padding": "0rem 0rem .15rem 2.7rem",
}

# NAVLINK AND ICONS
NAVLINK_STYLES={
    "padding": "0.5rem 1rem 0.5rem 2.7rem",
    "color": "white"
}
ICON1_STYLES={
    "position": "absolute",
    "padding": "12.75px 0 0 13px",
    "color": "white"
}
ICON2_STYLES={
    "position": "absolute",
    "padding": "55.75px 0 0 13px",
    "color": "white"
}
ICON3_STYLES={
    "position": "absolute",
    "padding": "98.75px 0 0 13px",
    "color": "white"
}
ICON4_STYLES={
    "position": "absolute",
    "padding": "139.75px 0 0 18px",
    "color": "white"
}
ICON5_STYLES={
    "position": "absolute",
    "padding": "365.75px 0 0 13px",
    "color": "white"
}
ICON6_STYLES={
    "position": "absolute",
    "padding": "394.75px 0 0 13px",
    "color": "white"
}
ICON7_STYLES={
    "position": "absolute",
    "padding": "423.75px 0 0 13.5px",
    "color": "white"
}
ICON8_STYLES={
    "position": "absolute",
    "padding": "453.75px 0 0 13.5px",
    "color": "white"
}
ICON9_STYLES={
    'padding': '23px',
    'color': '#0b65c2',
    "transform": "scale(1.4)"
}
ICON10_STYLES={
    'padding': '23px',
    'color': 'black',
    "transform": "scale(1.4)"
}
ICON11_STYLES={
    'padding': '23px',
    'color': 'red',
    "transform": "scale(1.4)"
}
# TOPIC VIS PAGE STYLES
TOPIC_VIS_CARD={
    "margin":"20px 40px",
}
TOPIC_PAGE_OUTER={
    "min-height":"95.5vh",
    "background": "rgb(255, 253, 251)"
}
TOPIC_PAGE_DESCRIPTION={
    "margin":"20px 40px"
}
TOPIC_DESCRIPTION_STYLES={
    "margin":"0",
    "padding": "20px"
}

# About page styles 
ABOUT_DESCRIPTION_STYLES={
    "margin":"0 0 20px 0",
    "padding": "20px 0"
}
ABOUT_DESCRIPTION_STYLES2={
    "margin":"35px 0 20px 0",
    "padding": "20px 0",
}
ABOUT_DESCRIPTION_STYLES3={
    "margin":"35px 0 20px 0",
    "padding": "20px 0",
    'text-align': 'center'
}
ABOUT_DESCRIPITON_LINES={
    "padding": "20px 40px"
}
ABOUT_LI_LINES={
    "padding": "5px 40px"
}
CONTACT_ME_DIVS_STYLES={
    'display': 'flex'
}

# Improvements on Main Page
MAIN_DISPLAY_CARD_STYLES={
  "width": "96.7%",
  "margin-left": "24px",
  "padding": "30px 0",
}

METRICS_LABEL_STYLES={
    "color": "grey",
    "padding": "15px 0 0 0",
    "font-size": "16px",
    "margin": "0",
}
METRICS_VALUE_STYLES={
    "color": "black",
    "font-size": "24px",
    "margin": "0",
    "padding": "7px 0",
}
YT_IFRAME_STYLES={
    "height": "300px", 
    "width": "100%", 
    "padding":"10px 20px", 
    # "margin-top": "1.4rem"
}


CYTO_LEGEND_STYLE={
    # "margin": "80px 20px 20px 55px"
}

VIS_PAGE_INSTRUCTION={
    "margin-top": "60px",
}
# Vis_PAGE_MENU STYLES
VIS_PAGE_MENU={
    'display': 'flex',
    'justify-content': 'space-around'
}
ANALYTICS_DROPDOWN={
    'width': '250px',
    'font-size': '1.3rem',
    "padding": "15px 0 0 0"
}


Vis_But_1={
    # "background": "#fff3eb",
    # "color": "black",
    # "height": "80px",
    "font-size": "1.2rem",
    "font-weight": "300",
    "width": "30%"
}
# VIS HEADING STYLES 
vis_headings={
    'text-align':'center',
    # 'margin-bottom': '10px',
    # 'letter-spacing':'1rem',
    
    
}
vis_subheadings={
    'font-size': '1.3rem',
    'text-align':'center',
    'margin-top': '18px',
    'padding': '5px'
}
heading_inside={
    
}
heading_outside={
    'padding': '45px 0',
    "height": "150px",
    'background': 'linear-gradient(180deg, #f4e5d6, transparent)',
    'color': 'black'
}

# LDA VIS STYLES 
ldavis_styles={
    # "padding": "10px 20px"
    "background":"white"
}
# NER VIS STYLES
NER_VIS_CARD_1={
    "width": "67%"
}
NER_VIS_CARD_2={
    "width": "32%"
}
NER_VIS_STYLES={
    "line-height": "2.2rem",
    "letter-spacing": "0.1rem"
}

NER_OUTER={
    "display":"flex",
    "justify-content": "space-between"
}

NER_VIS_1={
}
NER_VIS_2={
      # "width": "35%",
      "margin": "auto",
      "position":"sticky",
      "top": "32%"
}

# Gauge Styles 
GAUGE_STYLES={
    "margin-left": "28px"
}

# Visualizations Page CSS
VIS_PAGE_OUTER={
    "padding": "20px 40px",
    "min-height":"95.5vh",
    "background": "rgb(255, 253, 251)"
}


# Reccomendation Video Card Styles

VIDEO_ANALYTICS_DIV={
    "text-align": "center",
    "font-size": "36px",
    "color": "black",
    "font-weight": "200",
    "margin-bottom": "30px",
    "border-bottom": "1px solid #c9c9c9"
}
VIDEO_TITLE={
    "text-align": "center",
    "font-size": "36px",
    "color": "black",
    "font-weight": "200",
    "margin-bottom": "20px",
    "border-bottom": "1px solid #c9c9c9"
}
VIDEO_CARD_STYLE={
    "transition": "all 0.3s cubic-bezier(1, 0.05, 0.08, 0.73)",
    "background": "white",
    "margin": "36px 1.5rem",
    "border-radius": "1%",
    "padding": "10px 20px",
    "box-shadow": "rgb(0 0 0 / 19%) 0px 8px 10px 1px",
    "display": "none",
    "justify-content": "space-around"
}
RECOMMENDATION_VIDEO_CARD_STYLE={
    "transition": "all 0.3s cubic-bezier(1, 0.05, 0.08, 0.73)",
    "background": "white",
    "margin": "36px 1.5rem",
    "border-radius": "1%",
    "padding": "10px 20px",
    "box-shadow": "rgb(0 0 0 / 19%) 0px 8px 10px 1px",
    "display": "none",
    "justify-content": "space-around"
}
CARD_STYLE_INSIDE_1={
    "transition": "all 0.3s cubic-bezier(1, 0.05, 0.08, 0.73)",
    "border": "1px solid #ebebeb",
    "width": "125%",
    "padding": "20px 10px 10px 10px"
}
CARD_STYLE_INSIDE_2={
    "transition": "all 0.3s cubic-bezier(1, 0.05, 0.08, 0.73)",
    "border": "1px solid #ebebeb",
    "padding": "20px",
    "width": "100%"
}
CARD_STYLE_INSIDE_2_INSIDE={
    "display": "flex",
    "flex-direction": "column",
    "padding": "10x 5px",
    "text-align": "center",
    "border": "1px solid #ebebeb"
}
CARD_STYLE_INSIDE_2_INSIDE_SPL={
    "display": "flex",
    "flex-direction": "column",
    "margin-bottom": "35px",
    "padding": "10x 5px",
    "text-align": "center",
    "border": "1px solid #ebebeb"
}
CARD_STYLE_INSIDE_3={
    "transition": "all 0.3s cubic-bezier(1, 0.05, 0.08, 0.73)",
    "border": "1px solid #ebebeb",
    "padding": "20px 30px 0 30px",
    "width": "100%"
}
CARD_STYLE_INSIDE_4={
    # "border": "2px solid black",
    "width": "100%"
}
BUTTON_OUTER_DIV_STYLES={
    "text-align": "center",
    "margin": "25px 0 14px 0"
}
GO_BUTTON_STYLES={
    "width": "50%",
    "margin": "45px auto"
}

# Main Page Styles
MAIN_PAGE={
    "margin": "0px",
    # "width": "97rem",
    "background": "#fffdfb",
    # "color": "#fe6d01"
}
KT_DROPDOWN_STYLES={
    "width": "75%",
    "margin": "auto",
    "text-align": "center"
}
INSIDE_CONTENT_STYLES={
    "margin-top": "20px",
    "padding": "32px 60px",
    # "min-height": "602px",
    "min-height": "86vh",
    "font-size": "larger"
}
SIDEBAR_STYLE = {
    "position": "sticky",
    "float": "left",
    "z-index": "1",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "height": "123vh",
    "font-size": "large",
    "padding": "2rem 1rem",
    "background-color": "#000000"
}

CONTENT_STYLE = {
    "margin-left": "16rem",
    "margin-right": "0rem",
    # "padding": "2rem 1rem",
    "background-color": "rgb(255, 255, 255)",
    'font': {'family': 'Roboto'},
}
CARD_STYLE ={
    "margin": "0 0 0 180px"
}
FOOTER_INSIDE={
    "display": "flex",
    "justify-content": "space-around",
    "color": "black",
    "background": "linear-gradient(0deg, #f4e5d6, transparent)",
    "margin": "10px 0px 0px 0px",
    "padding": "20px 10px"
}
FOOTER_ROW ={
    "margin": "-23px 0 0 0",
    "padding": "0px"
}
FORM_STYLE={
        "padding": "20px 40px",
        "text-align": "center"
}
METRICS_STYLE={
    "'font': {'family': 'Roboto'},"
}

# 4. Dashboard Components

## 4.1 Sidebar

### 4.1.1 SideBar Indicators

In [39]:
"""
IN PROGRESS
This function returns the Dash/HTML to display gauges for #Video_Audio, #Keyword, #Topic

"""

def General_CreateGauge(text, num, paint = '13,110,253'): 
  fig = go.Figure(go.Indicator(
    mode = "number",
    value = num,
    number={"font":{"size":16, 'color':'white'}},
    title = {'text': text, "font":{"size":12, 'color':'white'}},
  ))
  fig.update_layout(width=int(70))
  fig.update_layout(height=int(56))
  fig.update_layout(paper_bgcolor='rgb('+paint+')')

  return dcc.Graph(figure=fig, style={"width": "4rem", "height":"3.75rem", "margin-right":"10px" }, config= {'displaylogo': False, 'displayModeBar': False})


"""
This function returns the Dash/HTML to display gauges for #videos, #topics, etc 

"""

def General_GetBasicInfo(): 
  numVideo = conn.getVertexCount("Video_Audio")
  numTopic = max(20, conn.getVertexCount("Topic"))
  numKeyword = conn.getVertexCount("Keyword")

  return dbc.Col(dbc.Row([
                  General_CreateGauge("Videos", numVideo,'0, 63, 89'),
                  General_CreateGauge("Topics", numTopic, '222, 102, 0'), 
                  General_CreateGauge("Keywords", numKeyword, '0, 122, 123')
                ],id="sidebar-indicators",style={'letter-spacing':'0.05rem', 'font-size': '11px', },))


In [40]:
sidebar = html.Div(
    [
        # A brief description 

        html.Center(html.P(
            "TigerGraph's Video Recommendation Dashboard", className="lead", style={'color':TG_YELLOW}
        )),
        html.Br(), 
     
        # Displaying the three gauges (num people, num movies, num reviews)

        html.Div(General_GetBasicInfo()),
        html.Hr(style = {'borderColor':WHITE}), 
     
        # The navbar itself, with three separate sections
        dbc.Nav(
            [                
                # inheriting icons directly below for resources through paddings
                html.I(className="fa-brands fa-medium", style=ICON5_STYLES),
                html.I(className="fa-brands fa-youtube", style=ICON6_STYLES),
                html.I(className="fa-brands fa-google", style=ICON7_STYLES),
                html.I(className="fa-brands fa-github", style=ICON8_STYLES),
                html.I(className="fa-solid fa-house", style=ICON1_STYLES),
                dbc.NavLink("General Overview", href="/", active="exact", style=NAVLINK_STYLES),
                html.I(className="fa-solid fa-chart-pie", style=ICON2_STYLES),
                dbc.NavLink("Explore Visuals", href="/analytics", active="exact", style=NAVLINK_STYLES),
                html.I(className="fa-solid fa-chart-area", style=ICON3_STYLES),
                dbc.NavLink("Topic Analytics", href="/topicanalytics", active="exact", style=NAVLINK_STYLES),
                html.I(className="fa-solid fa-info", style=ICON4_STYLES),
                dbc.NavLink("About", href="/about", active="exact", style=NAVLINK_STYLES),
            ],
            vertical=True,
            pills=True,
        ),
        
        html.P(
            "Enter keywords to pinpoint to its location in videos! 😄", style={'color':WHITE, 'margin-top': '22px'}
        ),     
        # External links that can be clicked on for further information 
     
        html.P("For more information, feel free to check out the following:", style={'color':WHITE, 'margin-top':'10px'}),
        html.P(html.A("Blog Post", href='https://btyagi.medium.com/video-recommendation-system-with-tigergraph-561dd60e563b', target="_blank", style={'color':WHITE}), style=RESOURCES_STYLES),
        html.P(html.A("Video Tutorial", href='https://www.youtube.com/watch?v=zkT9odf18rI', target="_blank", style={'color':WHITE}), style=RESOURCES_STYLES),
        html.P(html.A("Colab Notebook", href='https://github.com/atishaye/Video-Recommendation-System/tree/main/Google%20Colab%20Notebooks', target="_blank", style={'color':WHITE}), style=RESOURCES_STYLES),
        html.P(html.A("Repository", href='https://github.com/atishaye/Video-Recommendation-System', target="_blank", style={'color':WHITE}), style=RESOURCES_STYLES),  
        html.Br(), 
        html.Br(), 

        # The TigerGraph logo as well as a link to TG Cloud
     
        html.Center(dbc.Row(dbc.Col(html.Img(src=TG_LOGO, height="150px", style={"margin-bottom": "15px"})))),
        html.Center(html.B(html.A("TigerGraph Cloud", href="https://www.tigergraph.com/cloud/", target="_blank", style={'color':WHITE}))),

    ],
    id="sidebar-id",
    style=SIDEBAR_STYLE,
)

### 4.1.2 Title Card

In [41]:
titleCard =  dbc.Card([
                dbc.CardBody([
                              html.P(children="🎥/📽", className="header-emoji"),
                              html.H1(children="Video Recommendation System",
                                      className="header-title",),
                            ])
              ],
              color='danger', # Options include: primary, secondary, info, success, warning, danger, light, dark  
              inverse=True,   
              style={
                  "width":"60rem",
                  "margin-left":"1rem",
                  "margin-top":"1rem",
                  "margin-bottom":"1rem"
                  }
            )

## 4.2 Main Page | Dash General Overview Page

In [42]:
def Dash_GetGeneralPage():

    # numVideo = conn.getVertexCount("Video_Audio")
    # numTopic = max(20, int(conn.getVertexCount("Topic")))
    # numKeyword = conn.getVertexCount("Keyword")


    header = html.Div(
        children=[
            html.P(children="🎥", className="header-emoji"),
            html.H1(children="Video Recommendation System",
                    className="header-title",),
            html.P(
                children="Keyword based video recommendation that gives you the video sentiment & timestamp for description you're searching for",
                className="header-description",
            ),
        ],
        className="header",
    )
    form = html.Div(
        dbc.Form(
            dbc.Row(
                [
                    dbc.Col(
                        # dbc.Input(type="text", placeholder="What's in your mind today?"),
                        dcc.Dropdown(all_keywords_with_timestamps_list, placeholder = 'Choose or type a keyword to begin', id='demo-dropdown'), #all_keywords_with_timestamps_list[0],
                        className="mb-3",
                    ),

                    
                ],
                className="g-2",
            )
        ), style=FORM_STYLE
    )

    # titleH = html.Center(html.H1([
    #     "",
    #     dbc.Badge("Video Recommendation System", className="ml-1",
    #               style={'font-size': '60px'}, color="light"),
    # ],
    #     style={'color': "000000"},
    # ))

    # videosH = html.H3(["",
    #                    dbc.Badge(repr(numVideo), className="ml-1",
    #                              style={'font-size': '45px'}, color="primary"),
    #                    ])

    # topicsH = html.H3(["",
    #                    dbc.Badge(repr(numTopic), className="ml-1",
    #                              style={'font-size': '45px'}, color="info"),
    #                    ])

    # keywordH = html.H3(["",
    #                     dbc.Badge(repr(numKeyword), className="ml-1",
    #                               style={'font-size': '44px'}, color="secondary"),
    #                     ])
    

    footer = html.Div(
        children=[
            html.Pre(children="© 2022 TigerGraph | All Rights Reserved",
                     className="footer"),
            html.Pre(children="Made with ❤️ by TigerGraph Externs",
                     className="footer"),
        ],
        style=FOOTER_INSIDE,
    )


    # Creating the HTML element that will be returned
    row = html.Div([

        # The topmost row with the title
        # THIS IS HEADER
        dbc.Row([
            header,
            # width={"width":8,"offset":2}

        ],
        ),
        # THIS IS MAIN CONTENT INSIDE
        html.Div([
                  dbc.Card(dbc.CardBody([
                     
                  dbc.Row(
                [
                 
                    html.H3("Get your favorite video recommendations!", style={'color':'black', 'text-align':'center', "margin": "30px 0px"})
                    
                ],

            ),
            dbc.Row([
                form,
            ],
            ),
            # html.Hr(),

            dbc.Row([
                dbc.Col(
                    children=[dbc.Label("Video Title: "),
                    dcc.Dropdown(
                        id='video_dropdown',
                        placeholder='Enter a Video Title',
                        #options=list(keyword_to_video_lst.keys()),
                    ),],
                ),

                dbc.Col(
                    children=[
                    dbc.Label("Keyword Timestamp: "),
                    dcc.Dropdown(
                        id='timestamps',
                        placeholder='Enter a time to jump to',
                        #options=list(keyword_to_video_lst.values())[0]
                    ),],
                ),
                
            ], 
            style=KT_DROPDOWN_STYLES
            ),
            dbc.Col(
                    children=[
                        html.Div([
                          #Button
                          html.Div([
                                  dbc.Button("Go at Timestamp", id="Go-button", outline=True, size="lg", color="success", className="col-12"),
                          ]),
                      ],),
                    ],
                    style=GO_BUTTON_STYLES
                ),
               
                  ],
                  id="main-display-card-div"),
                  style=MAIN_DISPLAY_CARD_STYLES,
                  ),
                  html.Br(),
                  html.Br(),
            # VIDEO CARD TEMPLATE
            html.Div([
                html.Div([
                        # html.H3("Div2"),
                      html.Div([
                                #html.P("Video Title"),
                                html.P(id = 'title')
                      ],
                      style=VIDEO_TITLE
                      ),
                      html.Div([
                        html.Iframe(
                        id="yt",
                        src="",
                        style=YT_IFRAME_STYLES,
                        ),
                      ],
                      ),
                      html.Div([
                          #Button
                          html.Div([
                                  dbc.Button("Get Video Recommendations", id="Recommendation-button", color="success", className="col-11"),
                          ]),
                      ],
                      style={'text-align':'center', 'margin': '10px 0'}
                      ),
                      html.Div([
                          #Button
                          html.Div([
                                  dbc.Button("Get Video Visuals", href='/analytics' , id="Recommendation-Visualizations-button-0", color="danger", className="col-11",n_clicks=0),
                          ]),
                      ],
                      style={'text-align':'center'}
                      ),
                ],
                style=CARD_STYLE_INSIDE_1
                ),
                html.Div([
                          #html.H3("Div2"),
                          html.Div([
                            html.P("Video Analytics")
                          ],
                          style=VIDEO_ANALYTICS_DIV
                          ),
                          html.Div([
                               html.P('Total Number of keywords',style=METRICS_LABEL_STYLES),
                               html.P(id = 'num_word', style=METRICS_VALUE_STYLES)
                           ],
                           style=CARD_STYLE_INSIDE_2_INSIDE_SPL
                           ),
                           
                           html.Div([
                               html.P('Video Length',style=METRICS_LABEL_STYLES),
                               html.P(id = 'length', style=METRICS_VALUE_STYLES),
                           ],
                           style=CARD_STYLE_INSIDE_2_INSIDE_SPL
                           ),
                           
                           html.Div([
                               html.P('Video Theme',style=METRICS_LABEL_STYLES),
                               html.P(id = 'theme',style=METRICS_VALUE_STYLES)
                           ],
                           style=CARD_STYLE_INSIDE_2_INSIDE
                           ),

                ],
                style=CARD_STYLE_INSIDE_2
                ),
                html.Div([
                          html.Div([
                            html.P("Sentiment Score")
                          ],
                          style=VIDEO_ANALYTICS_DIV
                          ),
                          # html.H3("Div3"),
                          html.Div([
                              daq.Gauge(
                              id = 'gauge',
                              showCurrentValue=True,
                              color={"gradient":True,"ranges":{"#12c2e9":[-1,-0.8,], "#c471ed":[-0.8, 0.6], "#f64f59":[0.6,1]}},
                              value=0,
                              # label='Sentiment Score',
                              max=1,
                              size=280,
                              min=-1,
                            )
                          ],
                          style=GAUGE_STYLES
                          ),
                          
                ],
                style=CARD_STYLE_INSIDE_3
                ),
            
            ],
            id="First_Card_Div",
            style=VIDEO_CARD_STYLE
            ),

            dbc.Row(
                [
                    html.H3("⭐️ 🌟 ⭐️ Recommended Videos ⭐️ 🌟 ⭐️", style={'color':'black', 'text-align':'center', 'margin-top':'50px', 'display':'none'}, id='RV_TITLE')
                ],),
            html.Div([
                html.Div([
                        # html.H3("Div2"),
                      html.Div([
                          html.Div([
                          #html.P("Video Title"),
                          html.P(id = 'title1')
                          ],
                          style=VIDEO_TITLE
                          ),
                          html.Iframe(
                          id="rec1",
                          src="",
                          style=YT_IFRAME_STYLES,

                        ),          
                      ]),
                      html.Div([
                          #Button
                          html.Div([
                                  dbc.Button("Get Video Visuals", href='/analytics' , id="Recommendation-Visualizations-button-1", color="danger", className="col-11",n_clicks=0),
                          ]),
                      ],
                      style=BUTTON_OUTER_DIV_STYLES
                      ),
                ],
                style=CARD_STYLE_INSIDE_1
                ),
                html.Div([
                          # html.H3("Div2"),
                          html.Div([
                            html.P("Video Analytics")
                          ],
                          style=VIDEO_ANALYTICS_DIV
                          ),
                          html.Div([
                               html.P('Jaccard Similarity Score',style=METRICS_LABEL_STYLES),
                               html.P(id = 'score1',style=METRICS_VALUE_STYLES)
                           ],
                           style=CARD_STYLE_INSIDE_2_INSIDE
                           ),
                           
                           html.Div([
                               html.P('Total Number of keywords',style=METRICS_LABEL_STYLES),
                               html.P(id= 'num_word1' ,style=METRICS_VALUE_STYLES)
                           ],
                          style=CARD_STYLE_INSIDE_2_INSIDE

                           ),
                           
                           html.Div([
                               html.P('Video Theme',style=METRICS_LABEL_STYLES),
                               html.P(id = 'theme1',style=METRICS_VALUE_STYLES)
                           ],
                          style=CARD_STYLE_INSIDE_2_INSIDE
                                                     ),
                          html.Div([
                               html.P('Video Length',style=METRICS_LABEL_STYLES),
                               html.P(id = 'length1',style=METRICS_VALUE_STYLES)
                           ],style=CARD_STYLE_INSIDE_2_INSIDE),
                ],
                style=CARD_STYLE_INSIDE_2
                ),
                html.Div([
                          # html.H3("Div3"),
                          html.Div([
                            html.P("Sentiment Score")
                          ],
                          style=VIDEO_ANALYTICS_DIV
                          ),
                          html.Div([
                              daq.Gauge(
                              id = 'gauge1',
                              showCurrentValue=True,
                              color={"gradient":True,"ranges":{"#12c2e9":[-1,-0.8,], "#c471ed":[-0.8, 0.6], "#f64f59":[0.6,1]}},
                              value=0,
                              # label='Sentiment Score',
                              max=1,
                              min=-1,
                              size=280,
                            )          
                          ],
                          style=GAUGE_STYLES
                          ),
                          
                ],
                style=CARD_STYLE_INSIDE_3
                ),
            ],            
            className='Recommendation-Div',
            style=RECOMMENDATION_VIDEO_CARD_STYLE
            ),
            html.Div([
                html.Div([
                        # html.H3("Div2"),
                      html.Div([
                                #html.P("Video Title")
                                html.P(id = 'title2')
                      ],
                      style=VIDEO_TITLE
                      ),
                      html.Div([
                          html.Iframe(
                          id="rec2",
                          src="",
                          style=YT_IFRAME_STYLES,

                        ),          
                      ]),
                      html.Div([
                          #Button
                          html.Div([
                                  dbc.Button("Get Video Visuals", href='/analytics',id="Recommendation-Visualizations-button-2", color="danger", className="col-11",n_clicks=0),
                          ]),
                      ],
                      style=BUTTON_OUTER_DIV_STYLES
                      ),
                ],
                style=CARD_STYLE_INSIDE_1
                ),
                html.Div([
                           html.Div([
                            html.P("Video Analytics")
                          ],
                          style=VIDEO_ANALYTICS_DIV
                          ),
                           html.Div([
                               html.P('Jaccard Similarity Score',style=METRICS_LABEL_STYLES),
                               html.P(id = 'score2',style=METRICS_VALUE_STYLES)
                           ],
                                                     style=CARD_STYLE_INSIDE_2_INSIDE),
                           
                           html.Div([
                               html.P('Total Number of keywords',style=METRICS_LABEL_STYLES),
                               html.P(id= 'num_word2' ,style=METRICS_VALUE_STYLES)
                           ],
                                                     style=CARD_STYLE_INSIDE_2_INSIDE),
                           
                           html.Div([
                               html.P('Video Theme',style=METRICS_LABEL_STYLES),
                               html.P(id = 'theme2',style=METRICS_VALUE_STYLES)
                           ],style=CARD_STYLE_INSIDE_2_INSIDE),
                                                     
                           html.Div([
                               html.P('Video Length',style=METRICS_LABEL_STYLES),
                               html.P(id = 'length2',style=METRICS_VALUE_STYLES)
                           ],style=CARD_STYLE_INSIDE_2_INSIDE),
                ],
                style=CARD_STYLE_INSIDE_2
                ),
                html.Div([
                          html.Div([
                            html.P("Sentiment Score")
                          ],
                          style=VIDEO_ANALYTICS_DIV
                          ),
                          # html.H3("Div3"),
                          html.Div([
                              daq.Gauge(
                              id = 'gauge2',
                              showCurrentValue=True,
                              color={"gradient":True,"ranges":{"#12c2e9":[-1,-0.8,], "#c471ed":[-0.8, 0.6], "#f64f59":[0.6,1]}},
                              value=0,
                              # label='Sentiment Score',
                              max=1,
                              min=-1,
                              size=280,
                            )
                          ],
                          style=GAUGE_STYLES
                          ),
                ],
                style=CARD_STYLE_INSIDE_3
                ),
            
            ],
            className='Recommendation-Div',
            style=RECOMMENDATION_VIDEO_CARD_STYLE
            ),
                
            html.Div([
                html.Div([
                        # html.H3("Div2"),

                      html.Div([
                      html.Div([
                      #html.P("Video Title")
                      html.P(id = 'title3')
                      ],
                      style=VIDEO_TITLE
                      ),
                          html.Iframe(
                          id="rec3",
                          src="",
                          style=YT_IFRAME_STYLES,

                        ),          
                      ]),
                      html.Div([
                          #Button
                          html.Div([
                                  dbc.Button("Get Video Visuals" , href='/analytics', id="Recommendation-Visualizations-button-3", color="danger", className="col-11", n_clicks=0),
                          ]),
                      ],
                      style=BUTTON_OUTER_DIV_STYLES
                      ),
                ],
                style=CARD_STYLE_INSIDE_1
                ),
                html.Div([
                          html.Div([
                            html.P("Video Analytics")
                          ],
                          style=VIDEO_ANALYTICS_DIV
                          ),
                          # html.H3("Div2"),
                          html.Div([
                               html.P('Jaccard Similarity Score',style=METRICS_LABEL_STYLES),
                               html.P(id = 'score3',style=METRICS_VALUE_STYLES)
                           ],
                           style=CARD_STYLE_INSIDE_2_INSIDE),
                           
                           html.Div([
                               html.P('Total Number of keywords',style=METRICS_LABEL_STYLES),
                               html.P(id= 'num_word3' ,style=METRICS_VALUE_STYLES)
                           ],
                           style=CARD_STYLE_INSIDE_2_INSIDE),
                           
                           html.Div([
                               html.P('Video Theme',style=METRICS_LABEL_STYLES),
                               html.P(id = 'theme3',style=METRICS_VALUE_STYLES)
                               
                           ],
                           style=CARD_STYLE_INSIDE_2_INSIDE),
                          html.Div([
                               html.P('Video Length',style=METRICS_LABEL_STYLES),
                               html.P(id = 'length3', style=METRICS_VALUE_STYLES)
                           ],
                           style=CARD_STYLE_INSIDE_2_INSIDE),
                ],
                style=CARD_STYLE_INSIDE_2
                ),
                html.Div([
                          # html.H3("Div3"),
                          html.Div([
                            html.P("Sentiment Score")
                          ],
                          style=VIDEO_ANALYTICS_DIV
                          ),
                          html.Div([
                              daq.Gauge(
                              id = 'gauge3',
                              showCurrentValue=True,
                              color={"gradient":True,"ranges":{"#12c2e9":[-1,-0.8,], "#c471ed":[-0.8, 0.6], "#f64f59":[0.6,1]}},
                              value=0,
                              # label='Sentiment Score',
                              max=1,
                              min=-1,
                              size=280,
                            )
                          ],
                          style=GAUGE_STYLES
                          ),
                ],
                style=CARD_STYLE_INSIDE_3
                ),
            
            ],
            className='Recommendation-Div',
            style=RECOMMENDATION_VIDEO_CARD_STYLE
            ),
        ],
            style=INSIDE_CONTENT_STYLES,
        ),
        # THIS IS FOOTER
        html.Div(id='temp-div'),
        dbc.Row([
            footer
            # width={"width":8,"offset":2}
        ],
            style=FOOTER_ROW
        ),

    ],
    id="main-page",
        style=MAIN_PAGE
    )
    return row

 

### 4.2.1 Header

In [43]:
header=html.Div(
            children=[
                      html.Div(
                          children=[
                              html.P(children="🎥", className="header-emoji"),
                              html.H1(children="Video Recommendation System",
                                      className="header-title",),
                              html.P(
                                  children="Keyword based video recommendation that gives you the video sentiment & timestamp for description you're searching for",
                                  className="header-description",
                              ),
                          ],
                          className="header",
                      ),
            ],
            className='header-outer',
        )

### 4.2.2 Footer

In [44]:
footer=html.Div(
            children=[
                      html.Div(
                          children=[
                              html.Pre(children="© TigerGraph 2022 | All Rights Reserved", className="footer"),
                              html.Pre(children="Made with ❤️ by TigerGraph Externs", className="footer"),
                          ],
                          className="footer-div",
                      )
            ],
            className='footer-outer',
        )

## 4.3 Explore Visualizations Page

In [45]:
global titles
def visualizationsContent(index=-1):
    # MAIN VISUALIZATIONS DIV
    NERChart = dcc.Graph(
        id='NER_bar_chart',
        config={
        'displayModeBar': False
        }
    )
    footer = html.Div(
        children=[
            html.Pre(children="© 2022 TigerGraph | All Rights Reserved",
                     className="footer"),
            html.Pre(children="Made with ❤️ by TigerGraph Externs",
                     className="footer"),
        ],
        style=FOOTER_INSIDE,
    )

    cytoscape = html.Div([
                          dbc.Card(
                                    dbc.CardBody([
                                                  html.I(className="fa-solid fa-circle-question", id="target"),
                        html.Div(id = 'Cyto', className='eight columns', children=[
                                                                                   
                        ],
                        style=dict(width='100%')
                        ),

                        ],
                                        className="mb-3",
                                        #style={"width": "35rem"},
                                    ),
                                    outline=True,
                                    style={"width": '67%'},
                                ),

                        dbc.Card(
                                    dbc.CardBody([
                        
                        dbc.Container([tabs]+tooltips, style={"position": "relative"}),

                        html.Div([
                                  #Cyto Legend
                                  html.Div(
                                        id = 'cyto_legend',
                                        children=[
                                                html.Center("Legend", style={'font-size':'20px'}),
                                                cyto.Cytoscape(
                                                    id = 'legend_bool',
                                                    boxSelectionEnabled = False,
                                                    zoomingEnabled = False,
                                                    userPanningEnabled = False,
                                                    #responsive = True,
                                                    layout={'name': 'preset'},
                                                    #style={'width': '100%', 'height': '500px'},
                                                    stylesheet=default_cyto_stylesheet,
                                                    elements=[
                                                            {
                                                                'data': {'id': 'InitialNode', 'label': 'Theme'},
                                                                'position': {'x': 100, 'y': 100},
                                                                'locked': True,
                                                                'classes': 'genesis'
                                                            },
                                                            {
                                                                'data': {'id': 'TopicKey', 'label': 'Topic Keyword'},
                                                                'position': {'x': 100, 'y': 250},
                                                                'locked': True,
                                                                'classes': 'followingNode'
                                                            },
                                                            {
                                                                'data': {'id': 'VideoId', 'label': 'Video Name'},
                                                                'position': {'x': 250, 'y': 100},
                                                                'locked': True,
                                                                'classes': 'followerNode'
                                                            },
                                                            {'data': {'source': 'InitialNode', 'target': 'TopicKey'}, 'classes':'followingEdge'},
                                                            {'data': {'source': 'VideoId', 'target': 'InitialNode'}, 'classes':'followerEdge'},
                                                            {'data': {'source': 'VideoId', 'target': 'TopicKey'}, 'classes':'followerEdge'},
                                                    ],
                                                )
                                        ],
                                        #style= {'height':'600px', 'width':'50', 'position':'absolute', 'bottom':'0', 'right':'0',}
                                  ),
                        ],
                        style={ "flex-direction" : "column","position": "sticky","top": "45%","height": "30%"}
                        ),
                                    ],
                                        className="mb-3",
                                        #style={"width": "35rem"},
                                    ),
                                    outline=True,
                                    style={"width": "32%"},
                                ),
    ],
    style={"display":"flex", "justify-content": "space-between"}
    )
    initialTitle='astronomy'
    if(index!=-1):
      initialTitle=titles[index]

    return html.Div([
        ## VIS PAGE OUTER
        # HEADING
        html.Div([
                  html.Div([
                            html.H1("Explore Visuals", style=vis_headings, className='header-title'),      
                            html.P("Data Insights through visuals!",className="header-description",)
                  ],
                    style=heading_inside,      
                  ),
            
        ],
        style=heading_outside
        ),
        # VISUALIZATIONS_CONTENT
        html.Div([
                  dbc.Card(
                      dbc.CardBody([
                            html.Div([
                            html.Div([
                            html.H3("Select a Video", style=vis_subheadings),      
                  ]),
                            html.Div([
                            
                            dcc.Dropdown(
                                options= DropdownOptionsList(
                                    *list(vis_page_dropdown_list.keys())
                                    ), value=initialTitle, clearable = False, placeholder = 'Select a Video', id='analytics-dropdown'),

                        ],style=ANALYTICS_DROPDOWN),
                  dbc.Button(
                      "Transcript: Entity Recognition",
                      id="collapse-button",
                      className="",
                      color="secondary",
                      n_clicks=0,
                      outline=True,
                      style=Vis_But_1,
                  ),
                  dbc.Button(
                      "Topic Keywords Cytoscape",
                      id="collapse-button2",
                      className="",
                      color="secondary",
                      n_clicks=0,
                      outline=True,
                      style=Vis_But_1,
                  ),

                  ],id="VIS_PAGE_MENU",style=VIS_PAGE_MENU,
                  ),

                      ]),
                  ),
                  
                  
                  # CONTENT/Visualizations 
                  
                  dbc.Collapse(
                      html.Div(
                          [
                           html.Div([
                                # NER VIS for Transcripts
                                
                                dbc.Card(dbc.CardBody([
                                                       html.Div([
                                    
                                    html.Div(
                                      children="",#render(doc),
                                      id="NER_VIS_ID",
                                      style=NER_VIS_STYLES
                                    ),
                                ],
                                style=NER_VIS_1,
                                ),
                                ],),
                                style=NER_VIS_CARD_1
                                ),
                                # bar graph for NER
                                dbc.Card(dbc.CardBody([
                                                       
                                                       html.I(className="fa-solid fa-circle-question", id="ner_tooltip"),
                                                       
                                                       dbc.Tooltip([html.Li("Refer to the Bar Chart to view the various entities present in the transcript")],
                                                                   target="ner_tooltip", placement="bottom"),
                                                       html.Div([
                                                                 html.H3('Entity Bar Graph', style={'text-align': 'center', 'margin-bottom':'-50px'}),
                                          NERChart
                                ],
                                style=NER_VIS_2,
                                ),
                                ],),
                                style=NER_VIS_CARD_2
                                ),
                            ],
                            style=NER_OUTER,
                            ),
                          ],
                      ),
                      id="collapse",
                      is_open=False,
                  ),
                  

                    
      
                  
                  dbc.Collapse(
                      cytoscape,
                      id="collapse2",
                      is_open=False,
                  ),
                  # Instruction for Buttons
                  dbc.Card(
                      dbc.CardBody([
                          html.H4('Cytoscape & Entities What? Click on the buttons above to find out!', style={'text-align': 'center', 'margin': '0', 'padding':'20px 0'})          
                      ],),
                      style=VIS_PAGE_INSTRUCTION,
                  ),
                  
            
        ],
        style=VIS_PAGE_OUTER,
        ),
        # FOOTER
        html.Div([
              footer
        ],
        ),
    ],
    
    )

### 4.3.1 Cytoscape Components

#### 4.3.1.1 Utilites

In [46]:
def NamedDropdown(name, **kwargs):
    return html.Div(
        style={'margin': '10px 0px'},
        children=[
            html.P(
                [
                 html.Span(
                    f'{name}:',
                    id='cyto_dropdown_id',
                ),
                ],
                style={'margin-left': '3px'}
            ),

            dcc.Dropdown(**kwargs)
        ]
    )


def NamedRadioItems(name, **kwargs):
    return html.Div(
        style={'padding': '20px 10px 25px 4px'},
        children=[
            html.P(children=f'{name}:'),
            dcc.RadioItems(**kwargs)
        ]
    )


def NamedInput(name, **kwargs):
    return html.Div(
        children=[
            html.P(children=f'{name}:'),
            dcc.Input(**kwargs)
        ]
    )


# Utils
def DropdownOptionsList(*args):
    return [{'label': ' '.join(val.split('_')).title(), 'value': val} for val in args]
    
cyto_layouts = [
                'grid',
                'circle',
                'concentric',
                'breadthfirst',
                #'cose',
                'dagre',
                'cola',
                'klay',
                #'spread'
]

#### 4.3.1.2 Styles

In [47]:
default_cyto_stylesheet = [
    {
        "selector": 'node',
        'style': {
            "opacity": 0.65,
            'z-index': 9999,
            "label": "data(label)",
            "font-size": 12,
        }
    },
    {
        "selector": 'edge',
        'style': {
            "curve-style": "bezier",
            "opacity": 0.45,
            'z-index': 5000
        }
    },
    {
        'selector': '.followerNode',
        'style': {
            'background-color': '#0074D9',
            "text-opacity": 1,
        }
    },
    {
        'selector': '.followerEdge',
        "style": {
            "mid-target-arrow-color": "blue",
            "mid-target-arrow-shape": "vee",
            "line-color": "#0074D9"
        }
    },
    {
        'selector': '.followingNode',
        'style': {
            'background-color': '#FF4136',
            "text-opacity": 1,
        }
    },
    {
        'selector': '.followingEdge',
        "style": {
            "mid-target-arrow-color": "red",
            "mid-target-arrow-shape": "vee",
            "line-color": "#FF4136",
        }
    },
    {
        "selector": '.genesis',
        "style": {
            'background-color': 'white',
            "border-width": 2,
            "border-color": "black",
            "border-opacity": 1,
            "opacity": 1,

            "label": "data(label)",
            "color": "#000000",
            "font-size": 20,
            'z-index': 9999
        }
    },
    {
        'selector': ':selected',
        "style": {
            "border-width": 2,
            "border-color": "black",
            "border-opacity": 1,
            "opacity": 1,
            "label": "data(label)",
            "text-opacity": 1,
            "color": "black",
            "font-size": 13,
            'z-index': 9999
        }
    }
]
cyto_styles = {
    'json-output': {
        'overflow-y': 'scroll',
        'height': 'calc(50% - 25px)',
        'border': 'thin lightgrey solid'
    },
    'tab': {'height': 'calc(98vh - 80px)'}
}


#### 4.3.1.3 Tabs

In [48]:
tabs = dcc.Tabs(id='tabs', children=[
                            dcc.Tab(
                                NamedDropdown(
                                    name='Layout',
                                    id='dropdown-layout',
                                    options= DropdownOptionsList(
                                        'random',
                                        'grid',
                                        'circle',
                                        'concentric',
                                        'breadthfirst',
                                        'cose',
                                        'dagre',
                                        'cola',
                                        'klay',
                                        'spread',
                                        'euler'
                                    ),
                                    value="",
                                    clearable=False
                                ),
                                label='Control Panel',id = 'Control_Panel_tab',
                            ),

                            dcc.Tab(
                                html.Div(style=cyto_styles['tab'], children=[
                                    html.P('Node Object JSON:'),
                                    html.Pre(
                                        id='tap-node-json-output',
                                        style=cyto_styles['json-output']
                                    ),
                                    html.P('Edge Object JSON:'),
                                    html.Pre(
                                        id='tap-edge-json-output',
                                        style=cyto_styles['json-output']
                                    )
                                ]),
                                label='JSON', id = 'JSON_tab',
                        )
                    ])

#### 4.3.1.4 Tooltips

In [49]:
tooltips = [
 dbc.Tooltip([html.Li("Click on a node to expand its neighbours\n"),html.Li("Refer to the legend to understand the relation between two nodes"), html.Li("Scroll over it to change the size")],
            target="target"),

dbc.Tooltip("Select one of the options from the dropdown to change the Cyto's layout",
            target = "Control_Panel_tab",placement="bottom"),

dbc.Tooltip([html.Li("Click on a node to know more about it"), html.Li("Click on an edge to know more about it")],
            target="JSON_tab", placement="bottom"),
]

## 4.4 Topic Analytics Page

In [50]:
def topicVisPage():
  footer = html.Div(
    children=[
        html.Pre(children="© 2022 TigerGraph | All Rights Reserved",
                  className="footer"),
        html.Pre(children="Made with ❤️ by TigerGraph Externs",
                  className="footer"),
    ],
    style=FOOTER_INSIDE,
  )
  return html.Div([
                   # HEADING
                    html.Div([
                              html.Div([
                                  html.H1("Topic Analytics", style=vis_headings, className='header-title'),      
                                  html.P("Data Insights through Topic Modeling!",className="header-description",)
                        ],
                          style=heading_inside,      
                        ),
                        
                    ],
                    style=heading_outside
                    ),
                   dbc.Card(dbc.CardBody([
                        html.H5("Topic Modelling is a machine learning technique to categorise our transcripts into various topics through a machine learning model. Here is a fun way to visualise our corpus and keywords. Head over to our legend at the bottom to map what themes they belong to!", style=TOPIC_DESCRIPTION_STYLES)
                   ],),style=TOPIC_PAGE_DESCRIPTION),
                   dbc.Card(
                       dbc.CardBody([
                           html.Div([

                  html.Div([
                        html.I(className="fa-solid fa-circle-question", id="target"),
                                        dbc.Tooltip("The size of the bubbles indicate the importance of each topic in the whole corpus. The distance between the bubbles indicates the similarity between topics. On the top right we can adjust our relevance metric to get top 30 relevant words.",
                                                    target="target"),
                        
                        html.I(className="fa-solid fa-circle-question", id="relevance_tooltip", style = {"position": "absolute","left": "850px"}),
                        dbc.Tooltip([html.Li("Adjusting lambda to values close to 0 highlights potentially rare but more exclusive terms for the selected topic"),
                                     html.Li("Larger lambda values (closer to 1) highlight more frequently occurring terms in the document that might not be exclusive to the topic.")],
                                    target="relevance_tooltip", placement = 'bottom'),
                            
                        html.Iframe(
                        
                        src=app.get_asset_url("lda.html"),
                        style={"height": "130vh", "width": "100%", "padding":"5px"},
                        ),

                       dbc.ListGroup(
    [
        dbc.ListGroupItem("Theme",color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("Market", color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("Finance", color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("Crypto", color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("Crypto Currency", color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("Japanese Author", color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("Covid Effect", color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("Books", color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("Wildlife", color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("Animals", color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("Literature", color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("White Dwarf", color="#f6e9dc", style={'width':'20%'}),
        dbc.ListGroupItem("Bitcoin", color="#f6e9dc", style={'width':'20%'}),
     
    ],
    style={'display':'flex','flex-direction':'inherit'}
    ),
      dbc.ListGroup(
    [
        dbc.ListGroupItem("Topic Number", style={'width':'20%'}),
        dbc.ListGroupItem("1", style={'width':'20%'}),
        dbc.ListGroupItem("2", style={'width':'20%'}),
        dbc.ListGroupItem("3", style={'width':'20%'}),
        dbc.ListGroupItem("6", style={'width':'20%'}),
        dbc.ListGroupItem("7", style={'width':'20%'}),
        dbc.ListGroupItem("8", style={'width':'20%'}),
        dbc.ListGroupItem("9", style={'width':'20%'}),
        dbc.ListGroupItem("10", style={'width':'20%'}),
        dbc.ListGroupItem("14", style={'width':'20%'}),
        dbc.ListGroupItem("15", style={'width':'20%'}),
        dbc.ListGroupItem("17", style={'width':'20%'}),
        dbc.ListGroupItem("19", style={'width':'20%'}),
     
    ],
    style={'display':'flex','flex-direction':'inherit', 'margin': '0 0 20px 0'}
    ),
                      ],
                      id="pyldavis",
                      style=ldavis_styles
                      ),
                   ],style=TOPIC_PAGE_OUTER,
                   ),          
                       ],),
                       style=TOPIC_VIS_CARD
                   ),
                   # FOOTER
                  html.Div([
                        footer
                  ],
                  ),
  ])

## 4.5 About Page

In [51]:
def aboutContent():
  footer = html.Div(
    children=[
        html.Pre(children="© 2022 TigerGraph | All Rights Reserved",
                  className="footer"),
        html.Pre(children="Made with ❤️ by TigerGraph Externs",
                  className="footer"),
    ],
    style=FOOTER_INSIDE,
  )
  return html.Div([
                   # HEADING
                    html.Div([
                              html.Div([
                                  html.H1("About", style=vis_headings, className='header-title'),      
                                  html.P("How does this work?",className="header-description",)
                        ],
                          style=heading_inside,      
                        ),
                    ],
                    style=heading_outside
                    ),
                
                  #Main About Content
                  html.Div([
                            dbc.Card(
                                dbc.CardBody([
                                  html.H5('Welcome to the TigerGraph’s Video Recommendation Dashboard!', style=ABOUT_DESCRIPITON_LINES),
                                  html.H5('This is a Graph database based tool to get your favourite videos recommended based on provided keywords along with the ability to pinpoint where the keyword was spoken in the videos! The dataset incorporates a variety of themes like finance, cryptocurrency, animals, astronomy, literature etc for you to choose from.', style=ABOUT_DESCRIPITON_LINES),
                                  html.H5('The topics analytics page will allow you to find an interative topic model generated on the dataset that displays the themes and topics we have, and coordinates the similarity between different topics and the videos they belong to using Latent Dirichlet Allocation. ', style=ABOUT_DESCRIPITON_LINES),
                                  html.H5('Head over to our explore visuals section to dive deep into the dataset by looking at various entities and a dynamic cytoscape to visualise the network of keywords we are working with. You can observe how various keywords appear in different videos and the vast graph that connects them!',style=ABOUT_DESCRIPITON_LINES),
                                           ],
                                           style=ABOUT_DESCRIPTION_STYLES),
                                  style=ABOUT_DESCRIPTION_STYLES
                                ),
                            html.Br(),
                            html.Br(),
                            html.Div([
                                  html.H2("Contribute", style=vis_headings, className='header-title'),      
                                  html.P("How to contribute?",className="header-description",)
                              ],),
                              dbc.Card(
                                dbc.CardBody([
                                  html.H5('We realise that the initial content we created is just a starting point and we hope that the community can help in the journey refining and extending the contents.', style=ABOUT_DESCRIPITON_LINES),
                                  html.H5('Guidelines:', style=ABOUT_DESCRIPITON_LINES),
                                  html.Li('Should be locally tested and well formatted.', style=ABOUT_LI_LINES),
                                  html.Li('It is good practice to open an issue first and discuss your changes before submitting a pull request. This way, you can incorporate ideas from others before you even start.', style=ABOUT_LI_LINES),
                                  html.Li('GitHub Repository link is on the left Sidebar. Please follow the GitHub PR workflow for your contributions.',style=ABOUT_LI_LINES),
                                  html.Li('Fork this repo, create a feature branch, commit your changes and open a PR to this repo.',style=ABOUT_LI_LINES),
                                ],),
                                 style=ABOUT_DESCRIPTION_STYLES2
                            ),
                            html.Br(),
                            html.Br(),
                            html.Div([
                                  html.H2("Contact", style=vis_headings, className='header-title'),      
                                  html.P("How to contact?",className="header-description",)
                              ],),
                              dbc.Card(
                                dbc.CardBody([
                                              html.H5('You can always contact the externs below for any queries.', style=ABOUT_DESCRIPITON_LINES),
                                  html.Div([
                                            
                                      html.Div(
                                      [
                                          html.H5('Atishaye Jain', style=ABOUT_DESCRIPITON_LINES),
                                          html.A(html.I(className="fa-solid fa-envelope",style=ICON11_STYLES),href='mailto:atishaye2000@gmail.com', target='_blank'),
                                          html.A(html.I(className="fa-brands fa-linkedin",style=ICON9_STYLES),href='https://www.linkedin.com/in/atishaye-jain/', target='_blank'),
                                          html.A(html.I(className="fa-brands fa-github",style=ICON10_STYLES),href='https://www.github.com/atishaye', target='_blank'),
                                      ],
                                        style=CONTACT_ME_DIVS_STYLES,
                                    ),
                                    html.Div(
                                      [
                                          html.H5('Bhavya Tyagi', style=ABOUT_DESCRIPITON_LINES),
                                          html.A(html.I(className="fa-solid fa-envelope",style=ICON11_STYLES),href='mailto:bhavyatyagi16@gmail.com', target='_blank'),
                                          html.A(html.I(className="fa-brands fa-linkedin",style=ICON9_STYLES),href='https://www.linkedin.com/in/tyagibhavya/', target='_blank'),
                                          html.A(html.I(className="fa-brands fa-github",style=ICON10_STYLES),href='https://github.com/bhavyatyagi/', target='_blank'),
                                      ],
                                        style=CONTACT_ME_DIVS_STYLES,
                                    ),
                                    html.Div(
                                      [
                                          html.H5('Janhavi Lande', style=ABOUT_DESCRIPITON_LINES),
                                          html.A(html.I(className="fa-solid fa-envelope",style=ICON11_STYLES),href='mailto:landejanhavi23@gmail.com', target='_blank'),
                                          html.A(html.I(className="fa-brands fa-linkedin",style=ICON9_STYLES),href='https://www.linkedin.com/in/janhavi12/', target='_blank'),
                                          html.A(html.I(className="fa-brands fa-github",style=ICON10_STYLES),href='https://github.com/janhavilande', target='_blank'),
                                      ],
                                        style=CONTACT_ME_DIVS_STYLES,
                                    ),
                                  ],style={'display':'flex', 'justify-content': 'space-around'}),
                                ],),
                                style=ABOUT_DESCRIPTION_STYLES3
                            ),
                       
                   ],style=VIS_PAGE_OUTER,
                   ),
                   # FOOTER
                  html.Div([
                        footer
                  ],
                  ),
  ])

# 5. Dashboard Layout & Callbacks

In [52]:
app = JupyterDash(__name__,title='TG: Video Recommendation Dashboard',meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}], external_scripts=["https://drive.google.com/uc?export=view&id=1Oim_sXnuUW94YSVkPBgCISSDRF20h9Ox"], external_stylesheets=["https://drive.google.com/uc?export=view&id=1leGGNzs-FjfOPDlnD7hKfK_qA5lPkjfu", dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME], suppress_callback_exceptions=True)
content = html.Div(Dash_GetGeneralPage(), id="page-content", style=CONTENT_STYLE)

app.layout = html.Div(
    [dcc.Location(id="url"), sidebar, content,
    html.Div(id='dd-output-container'),visdcc.Run_js(id = 'javascript')])

# -------------------------------------------
# Collapsable divs on VIS PAGE
@app.callback(
    Output("collapse", "is_open"),
    [Input("collapse-button", "n_clicks")],
    [State("collapse", "is_open")],
)
def toggle_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

@app.callback(
    Output("collapse2", "is_open"),
    [Input("collapse-button2", "n_clicks")],
    [State("collapse2", "is_open")],
)
def toggle_collapse2(n, is_open):
    if n:
        return not is_open
    return is_open

# -------------------------------------------
@app.callback(Output("NER_VIS_ID", "children"),
              Output("NER_bar_chart", "figure"),
              [Input('analytics-dropdown', 'value')])
def ner(id):
    transcripts = conn.runInstalledQuery("get_transcripts", params={'vid': str(vis_page_dropdown_list[id])})
    # pprint(transcripts)
    df6 = conn.vertexSetToDataFrame(transcripts[0]['transcript'])
    # display(df6)
    transcript=str(df6['Text'][0])
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(transcript)
    d=NERUtils(transcript)

    countData = pd.DataFrame.from_dict(d, orient='index')
    colors = [DEFAULT_LABEL_COLORS[i] for i in d.keys()]

    fig = go.Figure(data=[go.Bar(
        x=list(d.values()), y=list(d.keys()),
        marker=dict(
            color=colors,
            line=dict(color='rgba(58, 71, 80, 1.0)', width=2)
        ),
        orientation='h'
    )])
    fig.update_layout(
        title="", # Adding a title
        yaxis_title="Entities", # Changing x-axis label
        xaxis_title="Count", # Changing y-axis label
        yaxis_tickangle=45,
        font=dict(
            size=16, # The font size
        ),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    #fig.show()

    return render(doc), fig

@app.callback(Output("video_dropdown", "options"), 
              [Input('demo-dropdown', 'value')], prevent_initial_call=True)

def update_video(value):
    global keyword_to_video, keyword_to_video_lst
    keyword_to_video = conn.runInstalledQuery("Keyword_to_VideoID", params={'k': value})
    keyword_to_video_lst = keyword_to_video[1]['@@vidName']
    keyword_to_video_dropdown_lst = sorted(list(keyword_to_video_lst.keys()))
    return DropdownOptionsList(*keyword_to_video_dropdown_lst)

@app.callback(Output("timestamps", "options"), 
              [Input('video_dropdown', 'value')], prevent_initial_call=True)
def update_timestamps(value):
    timestamps_in_float=list(map(float,keyword_to_video_lst[value]))
    timestamps_in_int=list(map(int,timestamps_in_float))
    timestamps_sorted=sorted(timestamps_in_int)
    return timestamps_sorted
titles=[]
@app.callback(Output("yt", "src"),
              Output("gauge", "value"),
              Output("num_word", "children"),
              Output("title", "children"),
              Output("length", "children"),
              Output("theme", "children"),
              [Input('video_dropdown', 'value'),
               Input('timestamps', 'value')], prevent_initial_call=True)
def display_timestamp(vid, ts = 0):
    global keyword_to_video_id
    titles.clear()
    keyword_to_video_id = keyword_to_video[0]['@@vidId'][vid]
    titles.append(get_title(keyword_to_video_id))
    yt_link = conn.getVerticesById("Video_Audio", str(keyword_to_video_id))
    yt_link = str(yt_link[0]['attributes']['Youtube_Link'])+'?start='+str(ts)

    c_sentiment = get_sentiments(keyword_to_video_id)['compound_sentiment']
    #return f'Output: {yt_link}'
    total_words = get_num_words(keyword_to_video_id)
    title = get_title(keyword_to_video_id)
    vid_len = get_time(keyword_to_video_id)

    return yt_link, c_sentiment, total_words, ' '.join(title.split('_')).title(), vid_len, themes[str(keyword_to_video_id)].title()

#Button Callback
@app.callback(Output("rec1", "src"),
              Output("rec2", "src"),
              Output("rec3", "src"),
              Output("gauge1", "value"),
              Output("gauge2", "value"),
              Output("gauge3", "value"),
              Output("num_word1", "children"),
              Output("num_word2", "children"),
              Output("num_word3", "children"),
              Output("score1", "children"),
              Output("score2", "children"),
              Output("score3", "children"),
              Output("title1", "children"),
              Output("title2", "children"),
              Output("title3", "children"),
              Output("length1", "children"),
              Output("length2", "children"),
              Output("length3", "children"),
              Output("theme1", "children"),
              Output("theme2", "children"),
              Output("theme3", "children"),
              [Input('Recommendation-button', 'n_clicks')], prevent_initial_call=True)
#callback func
def display_rec(num):
    rec = conn.runInstalledQuery("get_recommendation", params={'v_id':keyword_to_video_id})
    vid_arr = [i['Video_Audio'] for i in rec[0]['@@topResults']]
    jaccard_score = [i['tag'] for i in rec[0]['@@topResults']]
    res = conn.getVerticesById("Video_Audio", vid_arr)

    yt_link = []
    words = []
    
    c_sentiments = []
    vid_len = []
    
    for vid in vid_arr:
       titles.append(get_title(vid))
       c_sentiments.append(get_sentiments(vid)['compound_sentiment'])
       words.append(get_num_words(vid))
       vid_len.append(get_time(vid))


    for link in res:
        yt_link.append(link['attributes']['Youtube_Link'])
    
    

    return yt_link[0], yt_link[1], yt_link[2], c_sentiments[0], c_sentiments[1], \
        c_sentiments[2], words[0], words[1], words[2], \
        str(round(jaccard_score[0]*100,3))+' %', str(round(jaccard_score[1]*100,3))+' %', str(round(jaccard_score[2]*100,3))+' %', \
        make_title(titles[1]), make_title(titles[2]), make_title(titles[3]), \
        vid_len[0], vid_len[1], vid_len[2], \
        themes[str(vid_arr[0])].title(), themes[str(vid_arr[1])].title(), themes[str(vid_arr[2])].title()

@app.callback(
    Output('javascript', 'run'),
    [Input('main-page', 'n_clicks')])
def myfun(x): 
    if x: 
        return """
(function () {
    var list = document.getElementsByClassName("Recommendation-Div");
    var btn = document.getElementById('Recommendation-button');
    var gobtn = document.getElementById('Go-button');

    gobtn.addEventListener('click', function () {
        let sidebar = document.getElementById('sidebar-id');
        document.getElementById('First_Card_Div').style.display = "flex";
        window.scroll({
            top: 480,
            left: 0,
            behavior: 'smooth'
        });
        // sidebar.style.height = "1370px";
    });
    btn.addEventListener("click", function () {
        let sidebar = document.getElementById('sidebar-id');
        // sidebar.style.height = "2800px";
        window.scroll({
            top: 1180,
            left: 0,
            behavior: 'smooth'
        });
        document.getElementById("RV_TITLE").style.display = "block";
        for (let i = 0; i < list.length; i++) {
            var x = list[i];
            x.style.display = "flex";
        }
        document.getElementById('ldavis_el601398120692749607767031437').classList.add("mystyle");
    }, false);
}).call(this || {} || window);"""
    return ""

global initialVisualsIndex
initialVisualsIndex=-1
def setIndexValue(value):
  global initialVisualsIndex
  initialVisualsIndex=value
def getIndexValue():
  return initialVisualsIndex

@app.callback(Output("temp-div", "children"), 
              [Input("Recommendation-Visualizations-button-0", "n_clicks")],
              [Input("Recommendation-Visualizations-button-1", "n_clicks")],
              [Input("Recommendation-Visualizations-button-2", "n_clicks")],
              [Input("Recommendation-Visualizations-button-3", "n_clicks")],prevent_initial_call=True)
def render_page_content(btn1,btn2,btn3,btn4):
  changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
  if 'Recommendation-Visualizations-button-0' in changed_id:
      setIndexValue(0)
  elif 'Recommendation-Visualizations-button-1' in changed_id:
      setIndexValue(1)
  elif 'Recommendation-Visualizations-button-2' in changed_id:
      setIndexValue(2)
  elif 'Recommendation-Visualizations-button-3' in changed_id:
      setIndexValue(3)
  else:
      setIndexValue(-1)
  return html.Div("")

@app.callback(Output("page-content", "children"), 
              [Input("url", "pathname")],)

def render_page_content(pathname):

    if pathname == "/":
        return Dash_GetGeneralPage() 
    elif pathname == "/analytics":
        return visualizationsContent(getIndexValue())
    elif pathname == "/about":
        return aboutContent()
    elif pathname == "/topicanalytics":
      return topicVisPage()

    # If the user tries to reach a different page, return an error!
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P("Uh oh! Unfortunately, the pathname {pathname} was unable to be recognised..."),
        ]
    )

    
###############################################---<CYTOSCAPE>---#########################################################
@app.callback(Output("Cyto", "children"),
              Output("dropdown-layout", "value"),
              [Input('analytics-dropdown', 'value'),
               Input('collapse-button2', 'n_clicks')],
              [State("collapse2", "is_open")])
def call_cyto(id, click, is_open):
    if not click:
        return '', ''
    global following_node_di, following_edges_di, followers_node_di, followers_edges_di, default_elements, genesis_node

    edges = get_cyto_data(vis_page_dropdown_list[id])
    nodes = set()

    following_node_di = {}  # user id -> list of users they are following
    following_edges_di = {}  # user id -> list of cy edges starting from user id

    followers_node_di = {}  # user id -> list of followers (cy_node format)
    followers_edges_di = {}  # user id -> list of cy edges ending at user id

    cy_edges = []
    cy_nodes = []

    for edge in edges:

        source, target = edge[0], edge[1]
        #print(source, target)

        first = source
        sec = target
        first_label = source
        sec_label = target
        if '#' in source:
            first = source[0]+source[source.index('#')+1:]
        if '#' in target:
            sec = target[0]+target[target.index('#')+1:]
        edge_id = first+sec
        if 'Video' in source:
            first_label = rev_vis_page_dropdown_list[source[source.index('#')+1:]]
            first_label = ' '.join(first_label.split('_')).title()
        if 'Video' in target:
            sec_label = rev_vis_page_dropdown_list[target[target.index('#')+1:]]
            sec_label = ' '.join(sec_label.split('_')).title()
        if 'Topic' in source:
            first_label = themes[vis_page_dropdown_list[id]]
            first_label = (first_label+" (#"+source[source.index('#')+1:]+")").title()
        if 'Topic' in target:
            sec_label = themes[vis_page_dropdown_list[id]]
            sec_label = (sec_label+" (#"+target[target.index('#')+1:]+")").title()

        cy_edge = {'data': {'id': edge_id, 'source': source, 'target': target}}
        cy_target = {"data": {"id": target, "label": str(sec_label)}}
        cy_source = {"data": {"id": source, "label": str(first_label)}}

        if source not in nodes:
            nodes.add(source)
            cy_nodes.append(cy_source)
        if target not in nodes:
            nodes.add(target)
            cy_nodes.append(cy_target)

        # Process dictionary of following
        if not following_node_di.get(source):
            following_node_di[source] = []
        if not following_edges_di.get(source):
            following_edges_di[source] = []

        following_node_di[source].append(cy_target)
        following_edges_di[source].append(cy_edge)

        # Process dictionary of followers
        if not followers_node_di.get(target):
            followers_node_di[target] = []
        if not followers_edges_di.get(target):
            followers_edges_di[target] = []

        followers_node_di[target].append(cy_source)
        followers_edges_di[target].append(cy_edge)

    genesis_node = cy_nodes[1]
    genesis_node['classes'] = "genesis"
    default_elements = [genesis_node]

    Cyto = cyto.Cytoscape(
                            id='cytoscape',
                            elements=default_elements,
                            stylesheet=default_cyto_stylesheet,
                            autoRefreshLayout = True,
                            responsive = True,
                            minZoom = 0.5,
                            maxZoom = 3,
                            style={
                                'height': '120vh',
                                'width': '100%'
                            }

                        )

    return Cyto, random.choice(cyto_layouts)

###############################################---<EXPAND CYTO>---########################################################
@app.callback(Output('tap-node-json-output', 'children'),
              [Input('cytoscape', 'tapNode')])
def display_tap_node(data):
    return json.dumps(data, indent=2)


@app.callback(Output('tap-edge-json-output', 'children'),
              [Input('cytoscape', 'tapEdge')])
def display_tap_edge(data):
    return json.dumps(data, indent=2)


@app.callback(Output('cytoscape', 'layout'),
              [Input('dropdown-layout', 'value')])
def update_cytoscape_layout(layout):
    return {'name': layout, 'animate':True,}


@app.callback(Output('cytoscape', 'elements'),
              [Input('cytoscape', 'tapNodeData')],
              [State('cytoscape', 'elements')])
def generate_elements(nodeData, elements):
    if not nodeData:
        return default_elements

    followers_nodes = followers_node_di.get(nodeData['id'])
    followers_edges = followers_edges_di.get(nodeData['id'])

    if followers_nodes:
        for node in followers_nodes:
            node['classes'] = 'followerNode'
        elements.extend(followers_nodes)

    if followers_edges:
        for follower_edge in followers_edges:
            follower_edge['classes'] = 'followerEdge'
        elements.extend(followers_edges)

    following_nodes = following_node_di.get(nodeData['id'])
    following_edges = following_edges_di.get(nodeData['id'])

    if following_nodes:
        for node in following_nodes:
            if node['data']['id'] != genesis_node['data']['id']:
                node['classes'] = 'followingNode'
                elements.append(node)

    if following_edges:
        for follower_edge in following_edges:
            follower_edge['classes'] = 'followingEdge'
        elements.extend(following_edges)

    return elements

# 6. Run

In [53]:
app.run_server(mode='external', port = 5012)

Dash app running on:


<IPython.core.display.Javascript object>